See `README` file for more information about the process performed by this notebook.

In [1]:
import pandas as pd
import requests
from tqdm import tqdm 

import subprocess
import shutil
from multiprocessing import Pool
from pathlib import Path
from functools import partial
import re

In [2]:
# Path to clone repositories
DATA_DIR = Path('../data')
WORKING = Path('../data/actions')
WORKERS = 40

In [3]:
repositories_action = (
    pd.read_csv(DATA_DIR / 'action_name.csv.gz')
    [['action']]
    .values
    .tolist()
)


In [4]:
from collections import Counter 

In [5]:
a = [action[0].strip('/') for action in repositories_action]
print(len(Counter(a)))

2468


In [6]:
print(f'There are {len(repositories_action)} action repositories.')

There are 2472 action repositories.


Let's define a function that will do most of the job for a given repository and its default branch. Inline comments explain the process. 

In [28]:
def download_workflows(base_path: Path, repo: str, skip_existing=False, timeout=None):
    """
    Download the action.yml/yaml files for given repository. 
    
    :param base_path: target path for downloading the repository. 
    :param repo: name of the repository on GitHub. 
    :param branch: name of the branch to consider.
    :param timeout: timeout for individual command (in seconds).
    """
    # Create repository folder
    path = base_path / repo.replace('/', '---')
    try:
        path.mkdir(parents=True)
    except FileExistsError:
        if not skip_existing:
            raise
       
    # Check if given repository exists
    url = f'https://github.com/{repo}'
    r = requests.head(url)
    if r.status_code == 404:
        # Remove folder
        shutil.rmtree(path)
        return
    
    # Quick helper
    cmd = partial(subprocess.run, cwd=path, capture_output=True, text = True, timeout=timeout)
    
    # Initialize git repository
    out = cmd(['git', 'init'])
    
    # Configure git repository
    out = cmd(['git', 'remote', 'add', 'origin', 'https://github.com/' + repo])

    out = cmd(['git', 'config', 'core.sparsecheckout', 'true'])
    (path / '.git/info/sparse-checkout').write_text('action.yml\naction.yaml\nREADME.md\n')
    
    # Pull default branch
    try:
        branch = cmd(['git', 'remote', 'show', 'origin']).stdout      
        branch = re.search('HEAD branch: (.+)', branch).group(1)
        print(branch)
        out = cmd(['git', 'pull', 'origin', branch])
        out.check_returncode()
    except subprocess.CalledProcessError:
        # Remove folder
        shutil.rmtree(path)
        raise


Let's define a thin wrapper to handle errors.

In [29]:
def job(repository):
    repo = repository[0]
    try:
        download_workflows(WORKING, repo, skip_existing=True, timeout=None)
        return repo
    except Exception as e: 
        return repo, e

In [62]:
output = []
inputs = repositories_action[1800:2000]

with Pool(processes=WORKERS) as pool:
    jobs = pool.imap_unordered(job, inputs)
    for r in tqdm(jobs, total=len(inputs)):
        output.append(r)

  1%|▊                                                                                  | 2/200 [00:00<00:28,  6.97it/s]

master
main
main
master

  2%|█▏                                                                                 | 3/200 [00:00<00:57,  3.44it/s]


master
master
mastermastermaster


main
main
master
mastermaster

main
main
master
main
main
main
mastermastermaster


master
mastermain



  2%|██                                                                                 | 5/200 [00:00<00:34,  5.60it/s]

...

 94%|████████████████████████████████████████████████████████████████████████████▏    | 188/200 [00:05<00:00, 29.13it/s]

master


 98%|███████████████████████████████████████████████████████████████████████████████▍ | 196/200 [00:05<00:00, 38.25it/s]

main


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:05<00:00, 34.08it/s]


In [63]:
failures = [x for x in output if len(x) == 2]
print(f'There are {len(failures)} failing repositories out of {len(output)}.')
print('\n'.join(map(str, failures)))

There are 2 failing repositories out of 200.
('johnwbyrd/update-release', CalledProcessError(128, ['git', 'pull', 'origin', 'master'], '', "From https://github.com/johnwbyrd/update-release\n * tag               master     -> FETCH_HEAD\nfatal: update_ref failed for ref 'HEAD': cannot update ref 'refs/heads/master': trying to write non-commit object d2eeee14cc6eca1dde7b33127961c6fbf67670b5 to branch 'refs/heads/master'\n"))
('ljharb/actions-js-build/commit', AttributeError("'NoneType' object has no attribute 'group'"))
